In [ ]:
#import

import requests
import json
import pandas as pd
import hashlib
import random

In [ ]:
#블록 정보 조회
headers = {"Content-Type" : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)


In [ ]:
#transaction 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender" : "test_from",
        "recipient" : "test_to",
    "amount" : 3,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data = json.dumps(data)).content

In [ ]:
#채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)
print(res.text)

In [ ]:
#transaction2 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender" : "test_from",
    "recipient" : "test_to2",
    "amount" : 30,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

In [ ]:
#transaction3 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender" : "test_from",
    "recipient" : "test_to3",
    "amount" : 300,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

In [ ]:
#채굴하기
headers = {"Content-Type" : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
#노드의 블록 정보 확인 -4
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

In [ ]:
res.text

In [ ]:
#거래 내역 확인
status_json = json.loads(res.text)
status_json['chain']
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []


for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])


df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l
df_tx['sender'] = tx_sender_l
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l
df_tx

In [ ]:
#지갑별 잔액 계산
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended,columns = ['user', 'sended_amount']
df_received = pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_sended,columns = ['user', 'received_amount']
df_received



In [ ]:
df_status = pd.merge(df_received, df_sended, on = 'user', how= 'outer').fillna(0)
df_status['balance'] = df_status['received_amount'] - df_status['sended_amount']
df_status